# Ansatz Generation - Method by Ben / 1205.5707

In [ ]:
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division

import itertools
import re

In [ ]:
multiplicity = 5

all spinor brackets

In [ ]:
indices = list(range(1, multiplicity + 1))
pairs = list(itertools.combinations(indices, 2))
spas = ["⟨{}|{}⟩".format(*pair) for pair in pairs]
spbs = ["[{}|{}]".format(*pair) for pair in pairs]
sps = spas + spbs

In [ ]:
print(sps)

all schouten identities: ⟨ij⟩⟨kl⟩-⟨ik⟩⟨jl⟩+⟨il⟩⟨jk⟩=0

In [ ]:
quadruplets = list(itertools.combinations(indices, 4))
spas_schoutens = ["⟨{i}|{j}⟩⟨{k}|{l}⟩-⟨{i}|{k}⟩⟨{j}|{l}⟩+⟨{i}|{l}⟩⟨{j}|{k}⟩".format(**dict(zip(('i', 'j', 'k', 'l'), quadruplet))) for quadruplet in quadruplets]
spbs_schoutens = ["[{i}|{j}][{k}|{l}]-[{i}|{k}][{j}|{l}]+[{i}|{l}][{j}|{k}]".format(**dict(zip(('i', 'j', 'k', 'l'), quadruplet))) for quadruplet in quadruplets]
schoutens = spas_schoutens + spbs_schoutens
print(schoutens)

In [ ]:
len(schoutens)

all momentum conservation identities \Sum_i ⟨ji⟩[ik]

In [ ]:
mom_cons = []
for j in indices:
    for k in indices:
        mom_cons += ["".join(["+⟨{j}|{i}⟩[{i}|{k}]".format(i=i, j=j, k=k) if i not in (j, k) and j < i and i < k else
                              "-⟨{i}|{j}⟩[{i}|{k}]".format(i=i, j=j, k=k) if i not in (j, k) and i < j and i < k else
                              "+⟨{i}|{j}⟩[{k}|{i}]".format(i=i, j=j, k=k) if i not in (j, k) and i < j and k < i else
                              "-⟨{j}|{i}⟩[{k}|{i}]".format(i=i, j=j, k=k) if i not in (j, k) and j < i and k < i else
                              '' for i in indices])]
mom_cons = [mom_con if mom_con[0] != "+" else mom_con[1:] for mom_con in mom_cons]
print(mom_cons)

In [ ]:
len(mom_cons)

Sanity check

In [ ]:
import numpy

from lips import Particles

pA2 = re.compile(r'(?:\u27e8)(\d+)(?:\|)(\d+)(?:\u27e9)')
pA2bis = re.compile(r'(?:(?:\u27e8)(\d)(\d)(?:\u27e9))')
pS2 = re.compile(r'(?:\[)(\d+)(?:\|)(\d+)(?:\])')
pS2bis = re.compile(r'(?:\[)(\d)(\d)(?:\])')
pSijk = re.compile(r'(?:s_|S_)(\d+)')
pOijk = re.compile(r'(?:Ω_)(\d+)')
pPijk = re.compile(r'(?:Π_)(\d+)')
p3B = re.compile(r'(?:\u27e8|\[)(\d+)(?:\|\({0,1})([\d+[\+|-]*]*)(?:\){0,1}\|)(\d+)(?:\u27e9|\])')

def parse(string):
    string = string.replace("^", "**")
    string = pA2bis.sub(r"⟨\1|\2⟩", string)
    string = pA2.sub(r"oParticles.compute('⟨\1|\2⟩')", string)
    string = pS2bis.sub(r"[\1|\2]", string)
    string = pS2.sub(r"oParticles.compute('[\1|\2]')", string)
    string = pSijk.sub(r"oParticles.compute('s_\1')", string)
    string = pOijk.sub(r"oParticles.compute('Ω_\1')", string)
    string = pPijk.sub(r"oParticles.compute('Π_\1')", string)
    string = p3B.sub(r"oParticles.compute('⟨\1|(\2)|\3]')", string)
    string = re.sub(r'(\d)o', r'\1*o', string)
    string = string.replace(')o', ')*o')
    return string

In [ ]:
oParticles = Particles(int(multiplicity))

In [ ]:
assert all([numpy.isclose(complex(eval(parse(entry))), 0) for entry in mom_cons])
assert all([numpy.isclose(complex(eval(parse(entry))), 0) for entry in spas_schoutens])
assert all([numpy.isclose(complex(eval(parse(entry))), 0) for entry in spbs_schoutens])

converting to variables

In [ ]:
sps_vars = ['a{}'.format(i) for i in range(1, len(spas) + 1)] + ['b{}'.format(i) for i in range(1, len(spbs) + 1)]
sps_sub_dict, sps_inv_dict = dict(zip(sps, sps_vars)), dict(zip(sps_vars, sps))
def sps_repl(string):
    string = reduce(lambda x, y: x.replace(y, sps_sub_dict[y]), sps_sub_dict, string)
    string = re.sub(r"(\d)a", r"\1*a", string)
    string = re.sub(r"(\d)b", r"\1*b", string)
    return string

Groebner basis of ideal

In [ ]:
R = singular.ring(0, "(" + ", ".join(map(str, sps_vars)) + ")", 'dp')
R

In [ ]:
I = singular.ideal(list(map(sps_repl, schoutens)) + list(map(sps_repl, mom_cons)))
GB = I.groebner()
LTs = list(map(singular.lead, GB))
LTs

In [ ]:
len(LTs)

as exponents

In [ ]:
def to_exp(monomial, variables):
    """Converts a monomial into a list of exponents."""
    exps = numpy.zeros(len(variables), dtype=int)
    splitted_monomial = list(filter(lambda x: x != '', monomial.split("*")))
    for i, ientry in enumerate(splitted_monomial):
        if not ientry.isdigit():
            if i < len(splitted_monomial) - 1 and splitted_monomial[i + 1].isdigit():
                exp = int(splitted_monomial[i + 1])
            else:
                exp = 1
            exps[variables.index(ientry)] = exp
    return exps

def sps_to_exp(sps_monomial):
    return to_exp(sps_monomial, sps_vars)

In [ ]:
expLTs = list(map(sps_to_exp, map(str, LTs)))

In [ ]:
def exp_to_sps(exps):
    tuples = filter(lambda x: x[1] > 0, zip(sps_vars, exps))
    return "".join(["^".join([sps_inv_dict[tup[0]], str(tup[1])]) if tup[1] > 1 else sps_inv_dict[tup[0]] for tup in tuples])

phase weights

In [ ]:
def sps_var_to_pw(var):
    pw = numpy.zeros(multiplicity, dtype=int)
    spa_match = re.findall(r'⟨(\d)\|(\d)⟩', var)
    if spa_match != []:
        for i in spa_match[0]:
            pw[int(i) - 1] = 1
    else:
        spb_match = re.findall(r'\[(\d)\|(\d)\]', var)
        for i in spb_match[0]:
            pw[int(i) - 1] = -1
    return numpy.array(pw)

actual generation

In [ ]:
def basis_monomial_is_acceptable(exponents, expLTs):
    for expLT in expLTs:
        if min(numpy.array(exponents) - expLT) >= 0:
            return False
    else:
        return True

In [ ]:
def brute_force_recursion(md, pw, called_recursively=False):
    pw = numpy.array(pw)
    if md == 1 and sum(map(abs, pw)) == 2 and list(pw).count(0) == multiplicity - 2:
        i, j = (i + 1 for i, j in enumerate(pw) if j != 0)
        if min(pw) == 0:
            return [sps_to_exp(sps_sub_dict['⟨{}|{}⟩'.format(i, j)])]
        else:
            return [sps_to_exp(sps_sub_dict['[{}|{}]'.format(i, j)])]
    elif md <= 1:
        return []
    else:
        res = []
        for sps_var in sps_vars:
            if called_recursively is False:
                print('\rCurrent length', len(res), end='')
            var_pw = sps_var_to_pw(sps_inv_dict[sps_var])
            new_pw = pw - var_pw
            new_res = brute_force_recursion(md - 1, new_pw, called_recursively=True)
            sps_exp = sps_to_exp(sps_var)
            for entry in new_res:
                candidate = tuple(sps_exp + entry)
                if candidate not in res and basis_monomial_is_acceptable(candidate, expLTs):
                    res += [candidate]
        if called_recursively is False:
            print('\rFinal length', len(res), end='         ')
        return list(map(numpy.array, res))

In [ ]:
sps_vars[10]

In [ ]:
list(map(exp_to_sps, brute_force_recursion(2, [0, 0, 0, 0, 0])))

In [ ]:
list(map(exp_to_sps, brute_force_recursion(3, [-1, 0, -1, 0, 0])))

In [ ]:
%time list(map(exp_to_sps, brute_force_recursion(5, [1, 1, -1, 1, 0])))